In [1]:
from langchain_aws import ChatBedrockConverse
from langchain.schema import HumanMessage, SystemMessage
import boto3
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from opensearchpy import OpenSearch, RequestsHttpConnection
import os
import boto3
import json
import sys
from langchain.schema import BaseRetriever, Document
from typing import List, Optional, Dict, Tuple
from langchain.prompts.prompt import PromptTemplate
import uuid
from datetime import datetime
from botocore.exceptions import ClientError
from operator import itemgetter
from langchain_core.runnables import RunnableLambda
from boto3.dynamodb.conditions import Key
from pydantic import BaseModel, Field
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_aws import BedrockEmbeddings



In [2]:
print("rag")

rag


In [3]:
#bedrock client 설정
bedrock_client = boto3.client(
    service_name='bedrock-runtime',
    region_name='us-east-1'  # replace with your region
)

In [4]:
#embedding
bedrock_embeddings = BedrockEmbeddings(
    model_id="amazon.titan-embed-text-v2:0",  # You can choose other models like "cohere.embed-english-v3"
    region_name="us-east-1",  # Change to your AWS region
    client=bedrock_client  # Optional: provide your own boto3 client
)


In [5]:
#llm
llm = ChatBedrockConverse(
    model_id="anthropic.claude-3-sonnet-20240229-v1:0",
    client=bedrock_client,
    temperature=0.7,
    max_tokens=2000
)

In [6]:
loader = PyMuPDFLoader("/home/jovyan/langtest/langraph_work_shop/data/webzinepdf_6362.pdf")
docs = loader.load()
print(f"문서의 페이지수: {len(docs)}")

문서의 페이지수: 5


In [7]:
print(docs[1].page_content)


2024-14호│2024. 06. 25 ~ 07. 08
2
	
・디지털 기반의 서비스가 점차 복잡해지고 다양화되면서 사이버 공격은 매년 증가하고 있으
며, 특히 코로나 팬데믹을 겪은 2020년 이후 급격하게 증가한 디지털 활용으로 인해 사이버 
공격이 크게 증가
     <그림 1> 데이터 침해가 발생한 산업별 비중
        <그림 2> 사이버보안 사고 현황
자료: Kroll, 2024. 2. 7, Data breach outlook: 
Finance surpasses healthcare as most 
breached industry in 2023
자료: IMF
	
□금융 부문의 사이버보안 위험 사례가 증가하고 있는 가운데, 데이터 유출 및 시스템 공격, 피싱 등 
다양한 형태의 사이버보안 위협 문제가 나타남
	
— 금융기관은 다른 부문에 비해 대규모의 민감한 데이터를 보유하고 있어 해커들의 주요 표적이 
됨3)
	
・대규모 유출 사례로서 2019년 First American Financial의 8억 8,500만명의 고객 정보가 
유출된 사건이 있고, Equifax에서도 2017년 1억 6,000만명 이상의 민감한 정보가 유출됨
	
・2024년 들어서도 푸르덴셜 보험이 사이버 공격을 당하면서 250만명의 고객이 피해를 입었고 
LoanDepot에서도 1,600만명의 고객 데이터가 노출되는 등 계속적으로 중요한 정보에 대한 
데이터 침해 사고가 이어지고 있음
	
— 분산 서비스 거부 공격인 디도스(DDoS)는 대량의 악성 트래픽을 순간적으로 일으켜 서버를 마
비시키면서 사용자들의 접근 및 사용을 방해하는 것으로 금융권에서 빈번하게 발생
	
・2023년 전체 DDos공격의 35%가 금융 부문을 표적으로 삼으며 피해가 발생했으며 EMEA 
(유럽, 중동 및 아프리카) 지역에서 금융 부문에 대한 DDos 공격이 66%, 북미에서는 28%를 
차지4)
3)  Financial Times, 2024. 1. 16, Cyber attacks reveal fragi

In [8]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
split_documents = text_splitter.split_documents(docs)
print(f"분할된 청크의수: {len(split_documents)}")

분할된 청크의수: 17


In [9]:
vectorstore = FAISS.from_documents(documents=split_documents, embedding=bedrock_embeddings)


In [10]:
for doc in vectorstore.similarity_search("보안"):
    print(doc.page_content)

하여 사이버보안 위협에 효과적으로 대응해야 함
	
・IMF는 금융 부문의 사이버 위험 노출로 인한 손실은 심각한 시장 혼란이 발생할 수 있다고 
경고했고 특히 금융 부문은 민감한 데이터가 관련되어 있기 때문에 사이버 공격에 취약하다
는 것을 강조
	
・금융 부문의 보안 사고는 금융 시스템에 대한 신뢰를 약화시키고, 중요한 서비스를 방해하거
나, 다른 기관에 파급효과를 일으킬 경우 금융 및 경제 안정성을 위협하며, 금전적인 손실뿐 
아니라 평판 훼손과 같은 간접적인 손실도 상당히 높게 발생할 수 있음
	
— 금융기관도 사이버보안 전략 강화를 위한 적극적인 노력으로 보안 기술의 도입 및 직원 교육 등 
자체적인 보안 역량을 갖추는 것이 중요9)
	
・보안 모델 수립과 실시간 모니터링을 통한 침입감지 시스템, 데이터 암호화, 보안 교육 및 인식 
제고, 백업 및 복구 계획 등을 고려
종안을 발표하여 정교해진 사이버 공격에 대비한 사이버보안 프로그램의 개선과 감독을 강화
	
・유럽의 경우 데이터 보호 규정(GDPR), EU 사이버보안법, 네트워크 및 정보시스템 지침
(Network and Information Systems: NIS) 등을 통해 사이버보안을 강화하고 고객의 데이
터 보호 및 금융시스템 안정성 유지를 위한 지침 제시
	
・일본에서도 FSA는 금융기관 시스템에 대한 FISC(Financial Industry Information 
Systems) 보안 가이드라인을 개정하여 사이버보안을 더욱 강화할 것을 밝힘6)
	
— 국가 차원의 규제 및 가이드라인뿐 아니라 금융기관 자체에서도 사이버보안 위협을 방지하기 
위해 보안 전략을 강화하며 대응
	
・바클레이스는 고객정보보호와 거래 안전성 강화를 위해 사이버보안 및 회복성을 최우선 순
위로 삼고 사이버보안 전략을 수립
5)  IBM, 2024. 3. 17, What is phishing?
하고 복잡해진 기술로 인해 사이버보안 위험은 더욱 증가하는 상황
	
□금융기관은 민감한 정보 보호 및 금융시스템의 안정성, 

In [11]:
retriever = vectorstore.as_retriever(
    search_type="similarity",  # Options: "similarity", "mmr", or "similarity_score_threshold"
    search_kwargs={
        "k": 20  # Return top 5 most relevant documents

    }
)


In [12]:
prompt = PromptTemplate.from_template(
    """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Answer in Korean.

#Context: 
{context}

#Question:
{question}

#Answer:"""
)


In [13]:
security_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)


In [14]:
question = "고객데이터 관련 규정을 알려줘"
response = security_chain.invoke(question)
print(response)


국내에서는 고객 데이터 보호와 관련하여 개인정보보호법, 전자금융거래법, 전자금융감독규정 등의 법률이 마련되어 있습니다. 해당 법률은 금융기관의 고객 정보 유출 방지와 안전한 관리를 위한 규정을 포함하고 있습니다.


In [15]:
import boto3
import json

bedrock_agent_reranker = boto3.client('bedrock-agent-runtime', region_name="us-west-2")

modelId = "cohere.rerank-v3-5:0"
model_package_arn = f"arn:aws:bedrock:us-west-2::foundation-model/{modelId}"


In [16]:
def rerank_text(text_query, text_sources, num_results, model_package_arn):
    response = bedrock_agent_reranker.rerank(
        queries=[
            {
                "type": "TEXT",
                "textQuery": {
                    "text": text_query
                }
            }
        ],
        sources=text_sources,
        rerankingConfiguration={
            "type": "BEDROCK_RERANKING_MODEL",
            "bedrockRerankingConfiguration": {
                "numberOfResults": num_results,
                "modelConfiguration": {
                    "modelArn": model_package_arn,
                }
            }
        }
    )
    return response['results']

In [17]:
def reranker(query):
    text_sources = []
    retrieved_docs = retriever.get_relevant_documents(query)
    for i, doc in enumerate(retrieved_docs):
        text_sources.append({
            "type": "INLINE",
            "inlineDocumentSource": {
                "type": "TEXT",
                "textDocument": {
                    "text": doc.page_content,
                }
            }
        })
    response = rerank_text(query, text_sources, 5, model_package_arn)
    final_response = []
    for textid in response:
        final_response.append(text_sources[textid["index"]])
    return final_response

In [18]:
reranker("데이터 보호")

/tmp/ipykernel_6437/4189009193.py:3: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retrieved_docs = retriever.get_relevant_documents(query)


[{'type': 'INLINE',
  'inlineDocumentSource': {'type': 'TEXT',
   'textDocument': {'text': '종안을 발표하여 정교해진 사이버 공격에 대비한 사이버보안 프로그램의 개선과 감독을 강화\n\t\n・유럽의 경우 데이터 보호 규정(GDPR), EU 사이버보안법, 네트워크 및 정보시스템 지침\n(Network and Information Systems: NIS) 등을 통해 사이버보안을 강화하고 고객의 데이\n터 보호 및 금융시스템 안정성 유지를 위한 지침 제시\n\t\n・일본에서도 FSA는 금융기관 시스템에 대한 FISC(Financial Industry Information \nSystems) 보안 가이드라인을 개정하여 사이버보안을 더욱 강화할 것을 밝힘6)\n\t\n— 국가 차원의 규제 및 가이드라인뿐 아니라 금융기관 자체에서도 사이버보안 위협을 방지하기 \n위해 보안 전략을 강화하며 대응\n\t\n・바클레이스는 고객정보보호와 거래 안전성 강화를 위해 사이버보안 및 회복성을 최우선 순\n위로 삼고 사이버보안 전략을 수립\n5)\u2002 \x07IBM, 2024. 3. 17, What is phishing?'}}},
 {'type': 'INLINE',
  'inlineDocumentSource': {'type': 'TEXT',
   'textDocument': {'text': '2024-14호│2024. 06. 25 ~ 07. 08\n2\n\t\n・디지털 기반의 서비스가 점차 복잡해지고 다양화되면서 사이버 공격은 매년 증가하고 있으\n며, 특히 코로나 팬데믹을 겪은 2020년 이후 급격하게 증가한 디지털 활용으로 인해 사이버 \n공격이 크게 증가\n     <그림 1> 데이터 침해가 발생한 산업별 비중\n        <그림 2> 사이버보안 사고 현황\n자료: \x07Kroll, 2024. 2. 7, Data breach outlook: \nFinance surpasses healthcare as

In [19]:
security_chain_reranker = (
    {"context": itemgetter("question") | RunnableLambda(reranker) , 
     "question": itemgetter("question")}
    | prompt
    | llm
    | StrOutputParser()
)


In [20]:
question = "고객데이터 관련 규정을 알려줘"
response = security_chain_reranker.invoke({"question": question})
print(response)

고객 데이터 보호와 관련된 규정으로는 유럽연합의 데이터 보호 규정(GDPR)이 있습니다. GDPR은 고객의 데이터 보호 및 금융시스템의 안정성 유지를 위한 지침을 제시하고 있습니다. 또한 유럽에서는 EU 사이버보안법, 네트워크 및 정보시스템 지침(NIS) 등을 통해 사이버보안을 강화하고 고객 데이터를 보호하고 있습니다.
